In [1]:
import tensorflow as tf

In [2]:
tf.executing_eagerly()

True

In [7]:
a = tf.constant([[1, 3], [2, 4]])

In [8]:
b = tf.add(a, 2)

In [12]:
c = a * b
c

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 3, 15],
       [ 8, 24]], dtype=int32)>

In [16]:
m = tf.Variable([[1, 2], [3, 4]])
# Initialized immediately
c = tf.Variable([[3, 3], [3, 4]])

In [20]:
# No concept of a placeholder, since evaluation is immediate.
x = tf.Variable([[100, 200], [3, 4]])

In [22]:
y = m*x + c
y

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[103, 403],
       [ 12,  20]], dtype=int32)>

In [23]:
"""
Eager execution is
1. Easier to debug
2. Natural for python
3. Can use control flow; counters and loops
"""

'\nEager execution is\n1. Easier to debug\n2. Natural for python\n3. Can use control flow; counters and loops\n'

In [24]:
@tf.function
def add(a, b):
    return a + b

In [35]:
@tf.function
def matmul(a, b):
    print("goosfraba") # Will only execute once, during tracing
    return tf.matmul(a, b)

In [36]:
matmul([[1], [2]], [[3, 4]])

goosfraba


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3, 4],
       [6, 8]], dtype=int32)>

In [37]:
matmul([[1], [2]], [[3, 4]])

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3, 4],
       [6, 8]], dtype=int32)>

In [38]:
matmul([[1], [2]], [[3, 4]])

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3, 4],
       [6, 8]], dtype=int32)>

In [39]:
# Functions decorated can operate on Tensors, Variables, and Constants
@tf.function
def linear(m, x, c):
    return add(matmul(m, x), c)

In [44]:
m = tf.constant([[4.0, 5.0]])
m

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[4., 5.]], dtype=float32)>

In [45]:
x = tf.Variable([[3.0], [2.0]])
x

<tf.Variable 'Variable:0' shape=(2, 1) dtype=float32, numpy=
array([[3.],
       [2.]], dtype=float32)>

In [47]:
c = tf.constant([1.0])
c

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>

In [48]:
linear(m, x, c)

goosfraba


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[23.]], dtype=float32)>

In [49]:
linear(m, x, c)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[23.]], dtype=float32)>

In [63]:
@tf.function
def pos_neg_check(x):
    s = tf.reduce_sum(x)
    if s > 0:
        print('goobyboo')
        return 1
    elif s == 0:
        print('babaa')
        return 0
    else:
        print('yaolsd')
        return -1

In [64]:
pos_neg_check(tf.constant([100, 100]))
# Interesting to see that tf function decorater traced every branch on first execution!

goobyboo
babaa
yaolsd


<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [65]:
pos_neg_check(tf.constant([100, -100]))

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [66]:
pos_neg_check(tf.constant([-100, -100]))

<tf.Tensor: shape=(), dtype=int32, numpy=-1>

In [67]:
num = tf.Variable(7)

In [68]:
@tf.function
def add_time(x):
    for i in tf.range(x):
        num.assign_add(x)

In [69]:
add_time(5)

In [70]:
num

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=32>

In [74]:
a = tf.Variable(1.0)
b = tf.Variable(2.0)

In [75]:
@tf.function
def f(x, y):
    a.assign(y*b)
    b.assign_add(x*a)
    return a+b

In [76]:
f(a, b)

<tf.Tensor: shape=(), dtype=float32, numpy=22.0>

In [78]:
f(1, 2)

<tf.Tensor: shape=(), dtype=float32, numpy=270.0>

In [100]:
# Static graphs and polymorphism
@tf.function
def square(a):
    print('Tracing for', a.dtype)
    return a * a

In [101]:
square(tf.constant(2))

Tracing for <dtype: 'int32'>


<tf.Tensor: shape=(), dtype=int32, numpy=4>

In [102]:
# Re-traces a graph since we don't have square for float32 (so lazy,  haha)
square(tf.constant(2.0))

Tracing for <dtype: 'float32'>


<tf.Tensor: shape=(), dtype=float32, numpy=4.0>

In [103]:
square(tf.constant(3.0))
# Does not trace again, since float32 graph is available

<tf.Tensor: shape=(), dtype=float32, numpy=9.0>

In [104]:
square(tf.constant(3, dtype=tf.int64))
# Will be retraced for int64?

Tracing for <dtype: 'int64'>


<tf.Tensor: shape=(), dtype=int64, numpy=9>

In [105]:
# For each input type, a separate graph is retraced. Use `get_concrete_function` to get the types
concrete_int_square = square.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.int32))
concrete_int_square

Tracing for <dtype: 'int32'>


<ConcreteFunction square(a) at 0x149F82190>

In [106]:
concrete_float_square = square.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.float32))
concrete_float_square

Tracing for <dtype: 'float32'>


<ConcreteFunction square(a) at 0x149EA6A90>

In [107]:
concrete_float_square(tf.constant([2.0, 23.0]))

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([  4., 529.], dtype=float32)>

In [138]:
# Print seems confusing...
@tf.function
def p(x):
    print("Normal print", x)
    tf.print("TF Print", x)

In [139]:
p(1)

Normal print 1
TF Print 1


In [140]:
p(1)

TF Print 1


In [142]:
p("LOLO")

Normal print LOLO
TF Print LOLO


In [143]:
p("momo")

Normal print momo
TF Print momo


In [145]:
@tf.function
def func_with_variable_creation(x):
    y = tf.Variable([2.0])   # Need to initialize y outside the decorated function call
    return x * y

In [146]:
func_with_variable_creation(tf.constant([2.0]))

ValueError: in user code:

    <ipython-input-145-e0adca427ad3>:3 func_with_variable_creation  *
        y = tf.Variable([2.0])
    /Users/rajtilakindrajit/.virtualenvs/tf/lib/python3.7/site-packages/tensorflow/python/ops/variables.py:262 __call__  **
        return cls._variable_v2_call(*args, **kwargs)
    /Users/rajtilakindrajit/.virtualenvs/tf/lib/python3.7/site-packages/tensorflow/python/ops/variables.py:256 _variable_v2_call
        shape=shape)
    /Users/rajtilakindrajit/.virtualenvs/tf/lib/python3.7/site-packages/tensorflow/python/ops/variables.py:67 getter
        return captured_getter(captured_previous, **kwargs)
    /Users/rajtilakindrajit/.virtualenvs/tf/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py:702 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.


In [147]:
@tf.function
def func_with_variable_creation(x):
    y = tf.constant([2.0])
    return x * y

In [151]:
func_with_variable_creation(tf.constant([2.0]))

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.], dtype=float32)>

In [154]:
class C:
    def __init__(self):
        self.y = tf.Variable([2.0])

    @tf.function
    def func_with_variable_creation(self, x):
        return x * self.y

In [155]:
c = C()
c.func_with_variable_creation(tf.constant([2.0]))

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.], dtype=float32)>

In [159]:
def f(x):
    if x > 0:
        return 0
    else:
        return 1
print(tf.autograph.to_code(f))

def tf__f(x):
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal do_return, retval_
            (do_return, retval_) = vars_

        def if_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = 0
            except:
                do_return = False
                raise

        def else_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = 1
            except:
                do_return = False
                raise
        ag__.if_stmt((ag__.ld(x) > 0), if_body, else_body, get_state, set_state, ('do_return', 'retval_'), 2)
        return fsc

In [160]:
# tf.range generates a static graph, that can speed shit up!
@tf.function
def identity(x):
    return x

In [162]:
import time
s = time.time()
for i in tf.range(1000):
    identity(i)
e = time.time()
print(e-s)

0.24905920028686523


In [168]:
import logging
import warnings
warnings.filterwarnings('ignore')
logging.getLogger('tensorflow').disabled=True

In [169]:
import time
s = time.time()
for i in range(1000):
    identity(i)
e = time.time()
print(e-s)

3.6402742862701416
